In [31]:
from __future__ import division
from features import mfcc
from operator import add
import scipy.io.wavfile as wav
import numpy as np

words = ['a']


for x in range(len(words)):
    fileString = words[x]+"_mfcc"
    data = []
    for i in range(10):
                (rate,sig) = wav.read("training_sets/"+ words[x] + "-" + str(1) + ".wav")
                print ("Reading: " + words[x] + "-" + str(i+1) + ".wav")
                duration =len(sig)/rate
                mfcc_feat =mfcc(sig,rate)

                

    s = mfcc_feat[:20]
    st = []
    for elem in s:
            st.extend(elem)

            st //= np.max(np.abs(st),axis=0)
            data.append(st)
            print (st)

            with open("mfccData/" + fileString+ ".npy", 'w') as outfile:
                np.save(outfile,data)


Reading: a-1.wav


TypeError: 'float' object cannot be interpreted as an integer

In [28]:
sig

array([[  0,   0],
       [ -1,   0],
       [  0,   0],
       ...,
       [-27, -26],
       [  4,   4],
       [  2,   2]], dtype=int16)

In [20]:
rate

44100

In [18]:
sig

array([53, 57, 41, ..., -2, 22, 30], dtype=int16)

In [19]:
len(sig)

31232

In [38]:
rate

44100

In [4]:
def mfcc(signal,samplerate=16000,winlen=0.025,winstep=0.01,numcep=13,
          nfilt=26,nfft=512,lowfreq=0,highfreq=None,preemph=0.97,ceplifter=22,appendEnergy=True):
#     """Compute MFCC features from an audio signal.

#     :param signal: the audio signal from which to compute features. Should be an N*1 array
#     :param samplerate: the samplerate of the signal we are working with.
#     :param winlen: the length of the analysis window in seconds. Default is 0.025s (25 milliseconds)    
#     :param winstep: the step between successive windows in seconds. Default is 0.01s (10 milliseconds)    
#     :param numcep: the number of cepstrum to return, default 13    
#     :param nfilt: the number of filters in the filterbank, default 20.
#     :param nfft: the FFT size. Default is 512.
#     :param lowfreq: lowest band edge of mel filters. In Hz, default is 0.
#     :param highfreq: highest band edge of mel filters. In Hz, default is samplerate/2
#     :param preemph: apply preemphasis filter with preemph as coefficient. 0 is no filter. Default is 0.97. 
#     :param ceplifter: apply a lifter to final cepstral coefficients. 0 is no lifter. Default is 22. 
#     :param appendEnergy: if this is true, the zeroth cepstral coefficient is replaced with the log of the total frame energy.
#     :returns: A numpy array of size (NUMFRAMES by numcep) containing features. Each row holds 1 feature vector.
#     """            
    feat,energy = fbank(signal,samplerate,winlen,winstep,nfilt,nfft,lowfreq,highfreq,preemph)
    feat = numpy.log(feat)
    feat = dct(feat, type=2, axis=1, norm='ortho')[:,:numcep]
    feat = lifter(feat,ceplifter)
    if appendEnergy: feat[:,0] = numpy.log(energy) # replace first cepstral coefficient with log of frame energy
    return feat

In [5]:
def fbank(signal,samplerate=16000,winlen=0.025,winstep=0.01,
          nfilt=26,nfft=512,lowfreq=0,highfreq=None,preemph=0.97):
#     """Compute Mel-filterbank energy features from an audio signal.

#     :param signal: the audio signal from which to compute features. Should be an N*1 array
#     :param samplerate: the samplerate of the signal we are working with.
#     :param winlen: the length of the analysis window in seconds. Default is 0.025s (25 milliseconds)    
#     :param winstep: the step between seccessive windows in seconds. Default is 0.01s (10 milliseconds)    
#     :param nfilt: the number of filters in the filterbank, default 20.
#     :param nfft: the FFT size. Default is 512.
#     :param lowfreq: lowest band edge of mel filters. In Hz, default is 0.
#     :param highfreq: highest band edge of mel filters. In Hz, default is samplerate/2
#     :param preemph: apply preemphasis filter with preemph as coefficient. 0 is no filter. Default is 0.97. 
#     :returns: 2 values. The first is a numpy array of size (NUMFRAMES by nfilt) containing features. Each row holds 1 feature vector. The
#         second return value is the energy in each frame (total energy, unwindowed)
#     """          
    highfreq= highfreq or samplerate/2
    signal = sigproc.preemphasis(signal,preemph)
    frames = sigproc.framesig(signal, winlen*samplerate, winstep*samplerate)
    pspec = sigproc.powspec(frames,nfft)
    energy = numpy.sum(pspec,1) # this stores the total energy in each frame
    
    fb = get_filterbanks(nfilt,nfft,samplerate)
    feat = numpy.dot(pspec,fb.T) # compute the filterbank energies
    return feat,energy